In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Define gene lists
lpat_genes = ["LPAT1", "LPAT2", "LPAT3", "LPAT4"]
dgat_genes = ["DGATa", "DGATb", "DGATc", "DGATd", "DGATe", "DGATf", "DGATg", "DGAT2", "DGATh"]

# Define sample lists
samples = ["NIES_2145", "NIES_2146", "NIES_2145_bta1l"]

# Define Contigs dictionary
contigs = {
    "LPAT1": "JBEBFO010000001.1",
    "LPAT2": "JBEBFO010000006.1",
    "LPAT3": "JBEBFO010000010.1",
    "LPAT4": "JBEBFO010000010.1",
    "DGATa": "JBEBFO010000001.1",
    "DGATb": "JBEBFO010000001.1",
    "DGATc": "JBEBFO010000018.1",
    "DGATd": "JBEBFO010000002.1",
    "DGATe": "JBEBFO010000022.1",
    "DGATf": "JBEBFO010000004.1",
    "DGATg": "JBEBFO010000004.1",
    "DGAT2": "JBEBFO010000005.1",
    "DGATh": "JBEBFO010000008.1"
}

# 1. Load BED File

In [3]:
BED_colnames = ["chrom", "start_position", "end_position", "base_code", "score", "strand", "start_position2", "end_position2",
            "color", "Nvalid_cov", "percent_modified", "Nmod", "Ncanonical", "Nother_mod", "Ndelete", "Nfail", "Ndiff", "Nnocall"]

# Define sample gene BED file directory
bed_dir = "sample_gene_bed"

# Loop through each sample and gene, read .bed, store to variable and dict
for sample in samples:
    lpat_dict = {}
    dgat_dict = {}

    for gene in lpat_genes + dgat_genes:
        bed_path = os.path.join(bed_dir, f"{sample}_{gene}.bed")
        if os.path.exists(bed_path):
            bed_df = pd.read_csv(bed_path, sep="\t", header=None, names=BED_colnames)
            var_name = f"{sample}_{gene}"
            globals()[var_name] = bed_df

            # Add to LPAT or DGAT dict
            if gene in lpat_genes:
                lpat_dict[gene] = {"bed_df": bed_df, "contig": contigs[gene]}
            else:
                dgat_dict[gene] = {"bed_df": bed_df, "contig": contigs[gene]}

    # Store dictionaries globally (e.g. NIES_2145_LPAT)
    globals()[f"{sample}_LPAT"] = lpat_dict
    globals()[f"{sample}_DGAT"] = dgat_dict

# 2. Base Information for BED file

## 2.1. Sense base

In [4]:
# From bedtools getfasta, obtain sense base info for each base on each gene BED file, read as dataframe

reference_genome = "ref/Nanoce_C018.fna"
bed_dir = "sample_gene_bed"

for sample in samples:
    for gene in lpat_genes + dgat_genes:
        bed_path = f"{bed_dir}/{sample}_{gene}.bed"
        sensebase_bed_path = f"{bed_dir}/sensebase_{sample}_{gene}.bed"

        # Run bedtools getfasta
        !bedtools getfasta -fi {reference_genome} -bed {bed_path} -bedOut > {sensebase_bed_path}


## 2.2. Antisense base

In [5]:
def antisense(baseinfo_df):
    # Complementary base mapping
    complement_map = {'a': 't', 't': 'a', 'g': 'c', 'c': 'g',
                      'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G'}

    # Apply map directly on the column (vectorized, much faster)
    baseinfo_df["antisense_base"] = baseinfo_df["sense_base"].map(complement_map)

    return baseinfo_df

## 2.3. Combine base information

In [6]:
BED_sensebase_colnames = ["chrom", "start_position", "end_position", "base_code", "score", "strand", "start_position2", "end_position2",
            "color", "Nvalid_cov", "percent_modified", "Nmod", "Ncanonical", "Nother_mod", "Ndelete", "Nfail", "Ndiff", "Nnocall", "sense_base"]

In [7]:
for sample in samples:
    for gene in lpat_genes + dgat_genes:
        baseinfo = f"{sample}_{gene}_baseinfo"
        sensebase_bed_path = f"{bed_dir}/sensebase_{sample}_{gene}.bed"

        if os.path.exists(sensebase_bed_path):
            df = pd.read_csv(sensebase_bed_path, sep='\t', header=None, names=BED_sensebase_colnames)
            df_antisense = antisense(df)
            globals()[baseinfo] = df_antisense # This creates variables with sense and antisense base information, i.e., NIES_2145_LPAT1_baseinfo

# 3. Methylation motif (CpG, CHG, CHH) identification

## 3.1. Function to Identify Methylation Motifs (CpG, CHG, CHH)

In [8]:
import numpy as np

def find_methylation_motif(df_baseinfo):

    df_motifinfo = df_baseinfo.copy()
    # Remove duplicate C, which is 4mC
    df_motifinfo = df_motifinfo[df_motifinfo['base_code'] != '21839']
    # Remove duplicate base which has 6mA
    duplicate_base_1 = df_motifinfo['start_position'] == df_motifinfo['start_position'].shift(1)
    duplicate_base_min1 = df_motifinfo['start_position'] == df_motifinfo['start_position'].shift(-1)
    remove_duplicate_base = duplicate_base_1 & (df_motifinfo['base_code'] == 'a')
    remove_duplicate_base = duplicate_base_min1 & (df_motifinfo['base_code'] == 'a')
    df_motifinfo = df_motifinfo[~remove_duplicate_base].reset_index(drop=True)        

    # Create 'sense_motif' and 'antisense_motif' columns
    def classify_motif(base_seq, next_base1, next_base2):
        """Classifies the methylation motif based on base sequences."""
        if base_seq in ['C','c']:
            if next_base1 in ['G','g']:
                return 'CpG'
            elif next_base1 in ['A', 'a', 'T', 't', 'C', 'c']:
                if next_base2 in ['G', 'g']:
                    return 'CHG'
                elif next_base2 in ['A', 'a', 'T', 't', 'C', 'c']:
                    return 'CHH'
        return np.nan

    # Shift columns to access the next base(s)
    df_motifinfo['next_sense_base1'] = df_motifinfo['sense_base'].shift(-1)
    df_motifinfo['next_sense_base2'] = df_motifinfo['sense_base'].shift(-2)

    df_motifinfo['sense_motif'] = df_motifinfo.apply(
        lambda row: classify_motif(row['sense_base'], row['next_sense_base1'], row['next_sense_base2']), axis=1
    )

    df_motifinfo['next_antisense_base1'] = df_motifinfo['antisense_base'].shift(-1)
    df_motifinfo['next_antisense_base2'] = df_motifinfo['antisense_base'].shift(-2)

    df_motifinfo['antisense_motif'] = df_motifinfo.apply(
        lambda row: classify_motif(row['antisense_base'], row['next_antisense_base1'], row['next_antisense_base2']), axis=1
    )

    # Drop helper columns used for shifting
    df_motifinfo.drop(['next_sense_base1', 'next_sense_base2', 'next_antisense_base1', 'next_antisense_base2'], axis=1, inplace=True)

    return df_motifinfo

## 3.2. Execute function

In [31]:
for sample in samples:
    for gene in lpat_genes + dgat_genes:
        baseinfo = f"{sample}_{gene}_baseinfo"

        # Execute find motif info function
        if baseinfo in globals():
            motifinfo = f"{sample}_{gene}_motifinfo"
            globals()[motifinfo] = find_methylation_motif(globals()[baseinfo])

## 3.3. Filter base motif for each gene

In [33]:
motifs = ["CpG", "CHG", "CHH"]

for sample in samples:
    for gene in lpat_genes + dgat_genes:
        motifinfo = f"{sample}_{gene}_motifinfo"
        if motifinfo in globals():
            motifinfo = globals()[motifinfo]
            for motif in motifs:
                subset_var = f"{sample}_{gene}_motifinfo_{motif}"
                subset_df = motifinfo.loc[
                    ((motifinfo["sense_motif"] == motif) | (motifinfo["antisense_motif"] == motif)) &
                    (motifinfo["base_code"] == "m")
                ]
                globals()[subset_var] = subset_df

# 4. Methylation landscape per motif

In [ ]:
for sample in samples:
    for gene in lpat_genes + dgat_genes:
        for motif in motifs:
            contig = contigs[gene]
            sample_gene_motif = globals()[f"{sample}_{gene}_motifinfo_{motif}"]
            
            # Adjust scale based on the current gene dataframe
            x_axis_length = sample_gene_motif['end_position'].max() - sample_gene_motif['start_position'].min()

            # Let sns.relplot manage the figure creation
            g = sns.relplot(
                data=sample_gene_motif, x='start_position', y='percent_modified', size='score',
                kind='scatter', height=3, aspect=3.5,
                legend=True
            )

            # Set title and labels
            g.figure.suptitle(f"{sample}: {motif} Methylation in {gene} gene (Contig: {contig})", x=0.5, y=1.02)
            g.set_axis_labels("Base number", "% methylated base")
            g.set(xlim=(sample_gene_motif['start_position'].min(), sample_gene_motif['start_position'].min() + x_axis_length))
            g.set(ylim=(0,105))
            g._legend.set(title='Depth')

            # Save plot as image
            plt.savefig(f"visualization_motif_analysis/methylation_landscape/{sample}_{gene}_{motif}.png", bbox_inches='tight')


# 5. Major methylation type (whole genome level)

In [40]:
whole_genome_bed_files = [
    "Nanoce_2145_sorted.bed",
    "Nanoce_2146_sorted.bed",
    "Nanoce_2145_bta1l_sorted.bed"
]

reference_genome = "ref/Nanoce_C018.fna"

for bed_file in whole_genome_bed_files:
    !bedtools getfasta -fi {reference_genome} -bed nanno/bed/{whole_genome_bed_files} -bedOut > nanno/bed/sensebase_{whole_genome_bed_files}

zsh:1: bad pattern: nanno/bed/[Nanoce_2145_sorted.bed,
zsh:1: bad pattern: nanno/bed/[Nanoce_2145_sorted.bed,
zsh:1: bad pattern: nanno/bed/[Nanoce_2145_sorted.bed,
